In [21]:
import open3d as o3d
import numpy as np
import json
from glob import glob
import os
import cv2

# 1. Target PCD용 pcd 정합

### (1) 파일변환

In [7]:
rgb_paths = glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/color/*.jpg")
depth_paths = glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/depth/*.npy")
intr_paths = glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/intrinsics/*.json")
pose_paths = glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/poses/*.json")
cam2ee_path = "/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/ee2cam.json"

In [23]:
depth_npy_dir = "/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/depth"
output_dir = os.path.join(depth_npy_dir, "converted_png")
os.makedirs(output_dir, exist_ok=True)

converted_files = []
for npy_path in depth_paths:
    try:
        depth = np.load(npy_path).astype(np.uint16)
        filename = os.path.splitext(os.path.basename(npy_path))[0] + ".png"
        png_path = os.path.join(output_dir, filename)
        cv2.imwrite(png_path, depth)
        converted_files.append(png_path)
    except Exception as e:
        print(f"❌ {npy_path} 변환 실패: {e}")

converted_files[:5] 

['/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/depth/converted_png/2025-07-28_09-22-20.png',
 '/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/depth/converted_png/2025-07-28_09-20-49.png',
 '/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/depth/converted_png/2025-07-28_09-20-27.png',
 '/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/depth/converted_png/2025-07-28_09-20-20.png',
 '/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/depth/converted_png/2025-07-28_09-19-06.png']

In [26]:
def rgbd_to_pcd(rgb_path, depth_path, intrinsic_path, depth_scale=1000.0, depth_trunc=1.5):
    color_raw = o3d.io.read_image(rgb_path)
    depth_raw = o3d.io.read_image(depth_path)

    with open(intrinsic_path, "r") as f:
        intr_data = json.load(f)["color_intrinsics"]
        
    width = intr_data["width"]
    height = intr_data["height"]
    fx = intr_data["fx"]
    fy = intr_data["fy"]
    cx = intr_data["ppx"]
    cy = intr_data["ppy"]
        
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        width=width, height=height, fx=fx, fy=fy, cx=cx, cy=cy
    )

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_raw, depth_raw,
        depth_scale=depth_scale,
        depth_trunc=depth_trunc,
        convert_rgb_to_intensity=False
    )
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)
    return pcd

In [17]:
with open(intr_paths[0], "r") as f:
    intr_data = json.load(f)
    
intr_data["depth_intrinsics"]["width"]

848

### (2) 여러 프레임을 T_cam2base로 정렬하여 병합

In [4]:
def merge_pcds_with_transforms(pcd_list, transform_list):
    merged = o3d.geometry.PointCloud()
    for pcd, T in zip(pcd_list, transform_list):
        pcd_transformed = pcd.transform(T)
        merged += pcd_transformed
    return merged

### (3) T_cam2base 계산

In [10]:
with open(cam2ee_path, "r") as f:
    data = json.load(f)
R_ee2cam = np.array(data["R_ee2cam"])
t_ee2cam = np.array(data["t_ee2cam"]).reshape(3, 1)

T_ee2cam = np.eye(4)
T_ee2cam[:3, :3] = R_ee2cam
T_ee2cam[:3, 3] = t_ee2cam.flatten()

T_cam2ee = np.linalg.inv(T_ee2cam)

In [11]:
def pose_to_transform(pose_file):
    with open(pose_file, "r") as f:
        pose = json.load(f)
    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3, 1)
    T_base2ee = np.eye(4)
    T_base2ee[:3, :3] = R_base2ee
    T_base2ee[:3, 3] = t_base2ee.flatten()
    return T_base2ee @ T_cam2ee  # T_cam2base = T_base2ee * T_cam2ee

In [28]:
pcds = []
T_list = []
for rgb, depth, intr, pose in zip(rgb_paths, converted_files, intr_paths, pose_paths):
    try:
        pcd = rgbd_to_pcd(rgb, depth, intr)
        T = pose_to_transform(pose)
        pcds.append(pcd)
        T_list.append(T)
    except Exception as e:
        print(f"❌ Failed to load {rgb}: {e}")

if pcds:
    merged_pcd = merge_pcds_with_transforms(pcds, T_list)
    save_path = "./merged_target_pcd.ply"
    o3d.io.write_point_cloud(save_path, merged_pcd)
    result = f"✅ 평균 기준 PCD 저장 완료: {save_path}"
else:
    result = "❌ 변환 가능한 PCD가 없습니다."
    
result

'✅ 평균 기준 PCD 저장 완료: ./merged_target_pcd.ply'

# 2. 관찰 PC 생성

In [31]:
pcd = rgbd_to_pcd(rgb_paths[0], converted_files[0], intr_paths[0])
o3d.io.write_point_cloud("./camera_observed.ply", pcd)

True

# 3. ICP 최적화 (Point-to-Point ICP)

In [32]:
# === 초기 Tsai-Lenz 기반 ee2cam 변환행렬 불러오기 ===
T_ee2cam = np.eye(4)
T_ee2cam[:3, :3] = R_ee2cam  # Tsai-Lenz 결과
T_ee2cam[:3, 3] = t_ee2cam.flatten()

# === PCD 불러오기 ===
source_pcd = o3d.io.read_point_cloud("camera_observed.ply")  # EE 위치에서 관측한 것
target_pcd = o3d.io.read_point_cloud("merged_target_pcd.ply")  # 기준 Target

# === 초기 정렬 적용 ===
source_pcd.transform(T_ee2cam)

# === ICP 보정 ===
threshold = 0.01  # 1cm 허용 오차
icp_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    threshold,
    np.eye(4),
    o3d.pipelines.registration.TransformationEstimationPointToPoint()
)

# === 최종 보정 행렬 ===
T_icp = icp_result.transformation
T_ee2cam_refined = T_icp @ T_ee2cam

# === 최종 R, t 추출 ===
R_final = T_ee2cam_refined[:3, :3]
t_final = T_ee2cam_refined[:3, 3].reshape(3, 1)

In [34]:
result = {
    "R_cam2ee": R_final.tolist(),
    "t_cam2ee": t_final.flatten().tolist()  # 1D 리스트로 저장
}

# 저장
with open("optimized_cam2ee_icp.json", "w") as f:
    json.dump(result, f, indent=2)

# 4.Point-to-Plane ICP

In [37]:
# === ICP 실행 ===
threshold = 0.01  # 1cm
icp_result = o3d.pipelines.registration.registration_icp(
    source_pcd, target_pcd,
    threshold,
    np.eye(4),  # 이미 정렬했기 때문에 초기값은 identity
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)

# === 결과 확인 ===
print("✅ ICP fitness:", icp_result.fitness)
print("✅ ICP inlier RMSE:", icp_result.inlier_rmse)
print("✅ 보정 행렬 (delta T):")
print(icp_result.transformation)

✅ ICP fitness: 0.0
✅ ICP inlier RMSE: 0.0
✅ 보정 행렬 (delta T):
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [38]:
# === 최종 보정된 cam2ee = ΔT × 기존 T_cam2ee
T_refined = icp_result.transformation @ T_cam2ee
R_final = T_refined[:3, :3]
t_final = T_refined[:3, 3].reshape(3, 1)

# === JSON 저장
result = {
    "R_cam2ee": R_final.tolist(),
    "t_cam2ee": t_final.flatten().tolist()
}
with open("refined_cam2ee_icp.json", "w") as f:
    json.dump(result, f, indent=2)
print("✅ 저장 완료: refined_cam2ee_icp.json")

✅ 저장 완료: refined_cam2ee_icp.json


### 번외. PCD 시각화

In [39]:
def visualize_ply(filepath):
    """
    단일 PLY 파일을 시각화하는 함수
    """
    pcd = o3d.io.read_point_cloud(filepath)
    print(f"Loaded point cloud: {pcd}")
    o3d.visualization.draw_geometries([pcd])

def animate_ply_sequence(directory, num_frames, basename="frame", ext=".ply", delay=0.1):
    """
    연속된 ply 파일 시퀀스를 애니메이션으로 시각화하는 함수.
    예: frame0000.ply, frame0001.ply, ...
    - directory: 파일 경로
    - num_frames: 총 프레임 수
    - basename: 파일명 기준 이름
    - ext: 확장자
    - delay: 프레임 간 대기 시간 (초)
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    geometry = o3d.geometry.PointCloud()
    vis.add_geometry(geometry)

    for i in range(num_frames):
        filename = f"{directory}/{basename}{i:04d}{ext}"
        pcd = o3d.io.read_point_cloud(filename)
        geometry.points = pcd.points
        if pcd.has_colors():
            geometry.colors = pcd.colors
        vis.update_geometry(geometry)
        vis.poll_events()
        vis.update_renderer()
        o3d.utility.sleep(delay)

    vis.destroy_window()

In [62]:
visualize_ply("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/camera_observed.ply")

Loaded point cloud: PointCloud with 330758 points.
